In [1]:
import handcalcs.render
import forallpeople
from math import sqrt

forallpeople.environment('structural', top_level = True)
cm = 10*mm

# Modelo para cálculo de viga com seção compacta
Conhecer a viga é definir:
* Qual o momento de início do escoamento (yield moment) $0.7 M_y$?
    * $0.7 M_y=0.7 S F_y$
* Qual o momento de plastificação da seção $M_p$?
    * $M_p = Z f_y$

In [27]:
%%render 2
# Valores para W 610x125
S_x = 3241.30 * cm ** 3 # módulo elástico 
F_y = 350 * MPa
Z_x = 3697.30 * cm ** 3 # módulo plástico
M_y = S_x * F_y
M_esc =  0.7 * M_y
M_p = Z_x * F_y

<IPython.core.display.Latex object>

# Classificação da seção
* Qual a classificação da seção para flexão? - É isso que determina o tipo de E.L.U. que poderá atingir.
* O valor de $M_p$ acima só é alcançado caso a seção seja compacta.

In [3]:
%%render params
b_f = 229.0 * mm
t_f = 19.6 * mm
t_w = 11.9 * mm
d = 612.0 * mm
k_des = 16.0 * mm
h = d - 2*t_f - 2* k_des

<IPython.core.display.Latex object>

In [4]:
%%render 2
## Parâmetros de esbeltez:
lamb_f = b_f / (2 * t_f)
lamb_w = h/t_w

## Aço:
E = 210000 * MPa

<IPython.core.display.Latex object>

In [5]:
def sec_class(lamb_f,lamb_w,F_y):
    print('Obs.: De acordo com limites para seção monossimétrica:')
    # Limites para a flange
    lamb_p_f = 0.38 * sqrt(E/F_y)
    lamb_r_f = 1 * sqrt(E/F_y)
    #Limite para a alma
    lamb_p_w = 3.76 * sqrt(E/F_y)
    lamb_r_w = 5.7 * sqrt(E/F_y)
    
    if lamb_f < lamb_p_f:
        aba = 'compacta'
    elif lamb_p_f < lamb_f < lamb_r_f:
        aba = 'não_compacta'
    elif lamb_f > lamb_r_f:
        aba = 'esbelta'

    if lamb_w < lamb_p_w:
        alma = 'compacta'
    elif lamb_p_w < lamb_w < lamb_w_f:
        alma = 'não_compacta'
    elif lamb_w > lamb_r_w:
        alma = 'esbelta'
    
    print(f'- A alma é {alma}\n- A aba é {aba}')
        
sec_class(lamb_f, lamb_w, F_y)

Obs.: De acordo com limites para seção monossimétrica:
- A alma é compacta
- A aba é compacta


# Levar em conta o E.L.U. de F.L.T.
* O valor de $M_p$ só é atingido caso não ocorra F.L.T.:
    * Caso $L_b < L_p$, $M_n = M_p$, onde $L_b$ é o comprimento destravado.
    * Caso $L_p < L_b < L_r$, $ M_n = \text{interp. entre M_p e M_y}$
    * Caso $L_b > L_r$, $M_n = M_{cr}$ - E.L.U. de flambagem lateral torsional, M_cr é uma calculera

$$L_r = 1.95 r_{ts} \frac{E}{0.7F_y} \sqrt{\frac{Jc}{S_x h_o} + \sqrt{ \left( \frac{Jc}{S_x h_o}\right) ^2 + 6.76 \left( \frac{0.7 F_y}{E}\right)^2 }}$$

In [29]:
%%render
## Raio de giração:
r_y = 49.6 * mm
I_y = 4515.00 * cm ** 4
J = 159.50 * cm ** 4

h_o = d - t_f # Dist. entre centróides das flanges
r_ts = ((I_y*h_o)/(2*S_x))**0.5
c = 1 # para seção bissimétrica

L_r = 1.95*r_ts * (E/(.7*F_y)) * (J*c/(S_x*h_o)+((J*c/(S_x*h_o))**2 + 6.76*(.7*F_y/E)**2 )**.5)**.5

L_p = 1.76 * i_z * sqrt(E/F_y)

<IPython.core.display.Latex object>